### Deep learning model time!!!

<img src = pics/OIP.jpg width = 400>

In [2]:
import cupy
import numpy as np
import math
import time
import torch
cupy.cuda.set_allocator(None)       # no clue
from torch.utils.dlpack import from_dlpack

import numba
from numba import cuda

### Creating dataset
Each monte carlo simulation run is equivalent to one data point being made, so to generate a large dataset, we have to run monte carlo simulations lots of times, and batches can hypotheitcally make doing this faster.

here,the mc model from mc_snow, cuda version was imported and cleaned up a bit.
note that due to the existence of batches, some of the varaibles now need a bit of extra finagling to access properly. (s_0, Ki, Ko, mu, sigma, pot,r, d_normals, snowball_path_holder). Overall design is very close to original, though.

In [3]:
@cuda.jit               # defualt GPU
def monte_carlo_andtheholygrail_gpu(d_s, s_0, Ki, Ko, mu, sigma, pot,r,
                                    d_normals, snowball_path_holder, MONTHS,
                                    N_STEPS, N_PATHS, N_BATCH):
    

    # for shared memory (non)optimization
    # shared = cuda.shared.array(shape=0, dtype=numba.float32)
    # # load to shared memory
    # path_offset = cuda.blockIdx.x * cuda.blockDim.x

    # ii - overall thread index
    ii = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    stride = cuda.gridDim.x * cuda.blockDim.x

    for n in range(ii, N_PATHS * N_BATCH, stride):
        # newly added vars for N_BATCH calculations
        batch_id = n // N_PATHS
        path_id = n % N_PATHS       # equivalent to n in old code 

        snowball_path_holder[n][0] = s_0[batch_id]
        earlyexit = False
        ki = False
        mald = False
        for t in range(N_STEPS):
            # pre shared memory b_motion    
            #                                                   
            b_motion = d_normals[path_id + batch_id * N_PATHS +  t * N_PATHS * N_BATCH]

            # post shared memory b_motion
            # shared[cuda.threadIdx.x] = d_normals[path_offset + cuda.threadIdx.x + t * N_PATHS]

            dt = 1/N_STEPS
            # pre shared memory b_motion
            ds = snowball_path_holder[n][t] * mu[batch_id] * dt + snowball_path_holder[n][t] \
                                                * sigma[batch_id] * b_motion * math.sqrt(dt) 
            # post shared memory b_motion
            # ds = snowball_path_holder[n][t] * mu[batch_id] * dt + snowball_path_holder[n][t] * sigma[batch_id] * shared[cuda.threadIdx.x] * math.sqrt(dt) 
                    # no adjusting list sizes in cuda :(
            # snowball_path.append(snowball_path[t]+ds)
            snowball_path_holder[n][t+1] = snowball_path_holder[n][t] + ds
            

            # ki = snowball_path[t] + ds
            if snowball_path_holder[n][t+1] <= Ki[batch_id]:
                ki = True

            if not mald:
                for month in (0,1,2,3,4,5,6,7,8,9,10,11):                # need to do this instead because contains (in) and range are disabled
                    if t+1 == MONTHS[month]:     #startday no longer used to fake a start date in code
                        # price = t+1+startday
                        if snowball_path_holder[n][t+1] >= Ko[batch_id]:
                            price =  pot[batch_id] * t/365     # should turn t into int
                            # return snowball_path, price
                            d_s[n] =  price * math.exp(-r[batch_id] * t/N_STEPS)   # accounting for r
                            snowball_path_holder[n][-1] = d_s[n]            
                            earlyexit = True
                            mald = True
                            # print("blo got fucked\n")
                            break

        
        if not earlyexit:       # to prevent early exit getting out of bdds error
            # did not get knocked up or down
            price = pot[batch_id]
            # t  =T 
                        # CAN'T USE T CUZ CUDA IS FUCKING SHIT so use -1 instead
                        # or not ig T works now :sob:
            if ki and snowball_path_holder[n][N_STEPS] <= s_0[batch_id]:          # blo got knocked down and never recovered
                price = snowball_path_holder[n][N_STEPS] - s_0[batch_id]
            elif ki and snowball_path_holder[n][N_STEPS] <= Ko[batch_id]:          # blo got knocked down for a bit but finished above Ki
                price =0
            d_s[n] = price * math.exp(-r[batch_id])
            snowball_path_holder[n][-1] = d_s[n]    




However, for some reason, increasing the number of paths and/or the number of batches greatly slows down my monte's carlo's computational speed. I have no clue why this is the case, but would guess that i ran out of threads or gpus to run it on idk. Max len controls the number of data points, path controls how accurate each data point is. 

Once finished running, data is saved into a directory.

In [30]:
max_len = 1000                 # hundo thousand data points, final speed is ~40 min for 1000 data.
number_path = 500000
batch = 1
threads = 256
seed  =1999 
num = 0
max_length = max_len
N_PATHS = number_path
N_STEPS = 365
N_BATCH  =batch
# we will not be calculating a starting date since the difference is negligible and I aint rigging up
# a system to check if a certain day is a weekend or not
MONTHS = cupy.asnumpy([0, 31,59,90,120,151,181,212,243, 273,304,334])
        # SHOULD THIS BE NP ARRAY INSTEAD????
snowball_path_holder =  np.zeros(N_BATCH*N_PATHS, dtype=(np.float32,N_STEPS+1))# extra 1 is no longer for storing payoff
# self.snowball_path_holder = cupy.array(self.snowball_path_holder)
# self.T  = np.float(365.0)         # nah id lose. 
output = cupy.zeros(N_BATCH*N_PATHS, dtype = cupy.float32)
num_blocks  =(N_PATHS * N_BATCH -1) // threads +1
num_threads = threads


# Xs =  np.zeros(max_length, dtype=(np.float32,7))#
# Ys =  np.zeros(max_length, dtype=(np.float32))#          storing final data

Xss = []
Yss = []
# Xss = np.zeros((max_length, 7))
# Yss = np.zeros((max_length, 1))


# making sure self.snowball_path_holder is zeroed to avoid bug
# self.snowball_path_holder.fill(0)
s = time.time()

for i in range(max_length):
        randoms = cupy.random.normal(0,1, N_BATCH * N_PATHS * N_STEPS, dtype= cupy.float32)

        Xpre = cupy.random.rand(N_BATCH, 7, dtype = cupy.float32)
        #                        s_0,  Ki, Ko,  mu, sigma, pot, r
        Xpre = Xpre * cupy.array([4,  -2,  1,  .01,  .15,  10, .01], dtype=cupy.float32)
        X = Xpre +    cupy.array([8,   0,  0,  .02, .275,  15, .02], dtype=cupy.float32)
        # Ki and Ko will be set down here instead of the previous line to make them relative to s_0.
        X[:, 1] = X[:,0] -1         # overriding Ki and Ko 
        X[:, 2] = X[:,0] -.2        
        # print(X)
        X[:, 1] += Xpre[:,1]        # adding back the offset in Xpre after it gets overrided
        X[:, 2] += Xpre[:,2] 

        snowball_path_holder.fill(0)
                                        # d_s, s_0, Ki, Ko, mu, sigma, pot,r,
                                        # d_normals, snowball_path_holder, MONTHS,
                                        # N_STEPS, N_PATHS, N_BATCH):
        monte_carlo_andtheholygrail_gpu[(num_blocks,), (num_threads,)](
                                        output, X[:, 0], X[:, 1], X[:, 2], X[:, 3], 
                                        X[:, 4], X[:, 5], X[:, 6],
                                        randoms, snowball_path_holder, MONTHS,
                                        N_STEPS, N_PATHS, N_BATCH)
        # o = output.reshape(N_BATCH, N_PATHS)
        # Y  =o.mean(axis =1)         # getting the average of each batch
        Y = output.mean()
        X = X.mean(axis=0)
        Xss.append(X.tolist())
        Yss.append(Y.tolist())
        # Xss.append(X)
        # Yss.append(Y)
        
        # torch.save(from_dlpack(X.toDlpack()), f"snow_data/snowX_data/tensor{i}.pt")
        # torch.save(from_dlpack(Y.toDlpack()), f"snow_data/snowY_data/tensor{i}.pt")

        num+=1
        # print((from_dlpack(X.toDlpack()), from_dlpack(Y.toDlpack()))) 

v = output.mean()
cuda.synchronize()
e = time.time()
print('time', e-s, 'v', v, 'avg time', (e-s)/500000)

# Xs = np.array(Xss)
# Ys = np.array(Yss)
Xss = np.array(Xss)
Yss = np.array(Yss)
# print(Xss)
print(Yss)

tensorX = torch.Tensor(Xss)
tensorY = torch.Tensor(Yss)
print(tensorX)
print(tensorY)
torch.save(tensorX, "snow_data_tensor_test/tsnowX.pt")
torch.save(tensorY, "snow_data_tensor_test/tsnowY.pt")
# np.save("snow_npy_data/npyX", Xss)
# np.save("snow_npy_data/npyY", Yss)

c:\Users\xyzqadmin\anaconda3\Lib\site-packages\numba\cuda\cudadrv\devicearray.py:886: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


time 218.55252313613892 v 3.4649398 avg time 0.0004371050462722778
[2.48650241 2.55197453 3.61434937 3.18972516 2.29606295 3.6561799
 2.78525901 3.47168636 3.64172482 2.61491704 3.73917556 5.00415611
 2.76766586 2.2322309  2.52930498 3.97677541 2.48851228 2.0818131
 2.34652591 3.28007841 3.19151759 2.94404697 2.13764286 2.51589251
 2.93998623 2.34121871 3.71826243 3.09563994 2.11125541 2.91977859
 5.54066229 1.87857473 3.47983551 2.7077136  3.04440331 3.44855785
 5.86951065 3.08669853 3.00474715 2.90706038 2.74558449 3.30613899
 5.32979488 2.76495552 2.80513287 3.79385996 2.69700599 5.62145233
 3.52202106 2.23062658 2.68645096 2.62305379 2.48622751 2.19815993
 2.53211451 3.14271593 2.55764341 4.31880045 2.45253897 2.25858498
 3.41656899 1.91439164 3.07305551 2.43276644 4.2993145  2.91624856
 3.6151371  5.02526665 3.6280489  3.18084502 3.27265072 2.55380797
 2.46093941 2.59254909 2.26893759 2.08010554 2.56983185 2.29685831
 1.6850425  3.1227684  3.20789313 2.40722418 1.94819224 4.024037

Loading the data to confirm its existence! turning it into gpu monsters to prepare for throwing it into model to train! can prolly be done be done before saving, thouhg :/ <br>
oh well, it takes like no time to do

In [28]:
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torchvision import transforms, utils, datasets
import os

path = "snow_data_tensor"

def npy_loader(path):
    sample = torch.load(path)
    return sample

# dataset = datasets.DatasetFolder(
#     root=path,
#     loader=npy_loader,
#     extensions=['.pt']
#     )
# tensor_x = torch.Tensor(Xss)
# tensor_y = torch.Tensor(Yss)
tensor_x = torch.load("snow_data_tensor/tsnowX.pt")
tensor_y = torch.load("snow_data_tensor/tsnowY.pt")

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
inputs = tensor_x.float().to(device)
values = tensor_y.float().to(device)

# tdataset = TensorDataset(tensor_x,tensor_y) # create your datset
tdataset = TensorDataset(inputs,values) # create your datset

# print(tensor_x)

# print(tdataset)

# for i in tdataset:
# #     # print(i, "\n")
#     print(i)     # printing the Ys
# print(len(dataset)/2)
# print(dataset)
# inputs = tensor_x.float().to(device)
# # values = tensor_y.float().to(device)
# print(f"Input device is : cuda:{tensor_x.get_device()}")
# print(f"Target value device is : cuda:{tensor_y.get_device()}")

Now for some reason, the guy who made the nvidia notebook decided it was a good idea to make the dataset generate itself while training, making the training process significantly slower than if the data was already prepared already. This works, but is not that good. I would not reccomend, 2/5 stars.

In [ ]:

class SnowballDataSet(object):

    def __init__(self, max_len = 10, number_path = 1000, batch = 2, threads = 512, seed  =1999 ):
        self.num = 0
        self.max_length = max_len
        self.N_PATHS = number_path
        self.N_STEPS = 365
        self.N_BATCH  =batch
        # we will not be calculating a starting date since the difference is negligible and I aint rigging up
        # a system to check if a certain day is a weekend or not
        self.MONTHS = cupy.asnumpy([0, 31,59,90,120,151,181,212,243, 273,304,334])
                # SHOULD THIS BE NP ARRAY INSTEAD????
        self.snowball_path_holder =  np.zeros(self.N_BATCH*self.N_PATHS, dtype=(np.float32,self.N_STEPS+1))# extra 1 is no longer for storing payoff
        # self.snowball_path_holder = cupy.array(self.snowball_path_holder)
        # self.T  = np.float(365.0)         # nah id lose. 
        self.output = cupy.zeros(self.N_BATCH*self.N_PATHS, dtype = cupy.float32)
        self.num_blocks  =(self.N_PATHS * self.N_BATCH -1) // threads +1
        self.num_threads = threads

        #  temp_months, snowball_path_holder both added now
        cupy.random.seed(seed)

    def __len__(self):
        return self.max_length

    def __iter__(self):
        self.num = 0
        return self

    #   next basically takes the place of the cell running the mc. As such need to generate
     # (d_s, s_0, Ki, Ko, mu, sigma,  pot,r, d_normals, snowball_path_holder, MONTHS, N_STEPS, N_PATHS, N_BATCH
     # note that all but s_0, Ki, Ko, mu, sigma,  pot,r, d_normals have been generated in init due to their nonrandom nature
    def __next__(self):
        if self.num > self.max_length: 
            raise StopIteration      # nvidia notebook uses raise StopIteration here but p sure its deprecated???
                                      # is used because return returns an extra None
        # generating the variables
        # d_normals
        randoms = cupy.random.normal(0,1, self.N_BATCH * self.N_PATHS * self.N_STEPS, dtype= cupy.float32)

        Xpre = cupy.random.rand(self.N_BATCH, 7, dtype = cupy.float32)
        #                        s_0,  Ki, Ko,  mu, sigma, pot, r
        Xpre = Xpre * cupy.array([4,  -2,  1,  .01,  .15,  10, .01], dtype=cupy.float32)
        X = Xpre +    cupy.array([8,   0,  0,  .02, .275,  15, .02], dtype=cupy.float32)
        
        # Ki and Ko will be set down here instead of the previous line to make them relative to s_0.
        X[:, 1] = X[:,0] -1         # overriding Ki and Ko 
        X[:, 2] = X[:,0] -.2        
        # print(X)
        X[:, 1] += Xpre[:,1]        # adding back the offset in Xpre after it gets overrided
        X[:, 2] += Xpre[:,2] 

        # making sure self.snowball_path_holder is zeroed to avoid bug
        self.snowball_path_holder.fill(0)

                                        # d_s, s_0, Ki, Ko, mu, sigma, pot,r,
                                        # d_normals, snowball_path_holder, MONTHS,
                                        # N_STEPS, N_PATHS, N_BATCH):
        monte_carlo_andtheholygrail_gpu[(self.num_blocks,), (self.num_threads,)](
                                        self.output, X[:, 0], X[:, 1], X[:, 2], X[:, 3], 
                                        X[:, 4], X[:, 5], X[:, 6],
                                        randoms, self.snowball_path_holder, self.MONTHS,
                                        self.N_STEPS, self.N_PATHS, self.N_BATCH)
        
        o = self.output.reshape(self.N_BATCH, self.N_PATHS)
        Y  =o.mean(axis =1)         # getting the average of each batch
        self.num+=1
        return (from_dlpack(X.toDlpack()), from_dlpack(Y.toDlpack()))



And now a small test run.

In [ ]:
# ds = SnowballDataSet(10, number_path=500000, batch=16, seed=15)
ds = SnowballDataSet(10, number_path=500000, batch=1, seed=15)
for i in ds:
    # print(i, "\n")
    print(i[0],i[1])     # printing the Ys

c:\Users\xyzqadmin\anaconda3\Lib\site-packages\numba\cuda\cudadrv\devicearray.py:886: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


tensor([[ 9.6349,  7.2834,  9.7545,  0.0287,  0.3059, 24.2591,  0.0276]],
       device='cuda:0') tensor([3.9449], device='cuda:0')
tensor([[ 8.6179,  6.7289,  8.7578,  0.0208,  0.2911, 19.6906,  0.0254]],
       device='cuda:0') tensor([3.1118], device='cuda:0')
tensor([[ 8.5157,  5.8327,  8.6394,  0.0262,  0.2960, 21.1468,  0.0275]],
       device='cuda:0') tensor([4.7028], device='cuda:0')
tensor([[ 8.8283,  7.1150,  8.9113,  0.0208,  0.3009, 18.3316,  0.0227]],
       device='cuda:0') tensor([2.5807], device='cuda:0')
tensor([[11.8870,  9.9710, 11.7828,  0.0285,  0.3634, 23.8967,  0.0244]],
       device='cuda:0') tensor([2.9223], device='cuda:0')
tensor([[11.4234,  9.4665, 11.8392,  0.0211,  0.3384, 19.8409,  0.0246]],
       device='cuda:0') tensor([2.6680], device='cuda:0')
tensor([[11.0768,  8.6847, 11.8625,  0.0211,  0.3254, 20.9236,  0.0287]],
       device='cuda:0') tensor([3.6412], device='cuda:0')
tensor([[11.1044,  9.8112, 11.2277,  0.0231,  0.3005, 19.0593,  0.0228]],
  

### Creating the model

Erm pretty default model. Just making it have functionality. normalizing it accoriding to the average value of all of the input variables.

In [31]:
%%writefile snow_model.py
import torch.nn as nn
import torch.nn.functional as F
import torch


class Net(nn.Module):

    def __init__(self, hidden=1024):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(7, hidden)
        self.fc2 = nn.Linear(hidden, hidden)
        self.fc3 = nn.Linear(hidden, hidden)
        self.fc4 = nn.Linear(hidden, hidden)
        self.fc5 = nn.Linear(hidden, hidden)
        self.fc6 = nn.Linear(hidden, hidden)
        self.fc7 = nn.Linear(hidden, 1)
        self.register_buffer('norm',
                             torch.tensor([10.0,
                                           8.5,
                                           10.4,
                                           0.025,
                                           0.35,
                                           0.20,
                                           0.025]))

    def forward(self, x):
        # normalize the parameter to range [0-1] 
        x = x / self.norm
        x = F.elu(self.fc1(x))
        x = F.elu(self.fc2(x))
        x = F.elu(self.fc3(x))
        x = F.elu(self.fc4(x))
        x = F.elu(self.fc5(x))
        x = F.elu(self.fc6(x))
        return self.fc7(x)


Overwriting snow_model.py


### Running the model
YAP CENTRAL

In [29]:
from ignite.engine import Engine, Events
from ignite.handlers import Timer
from torch.nn import MSELoss
from torch.optim import Adam
from ignite.contrib.handlers.param_scheduler import CosineAnnealingScheduler
from ignite.handlers import ModelCheckpoint

from snow_model import Net
# from snow_model_module import Net
# from snow_model_2 import Net

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Device used : {device}")

# from cupy_dataset import OptionDataSet
timer = Timer(average=True)
model = Net().cuda()
loss_fn = MSELoss()
optimizer = Adam(model.parameters(), lr=1e-3)
# dataset = OptionDataSet(max_len=10000, number_path = 1024, batch=4800)
# dataset = SnowballDataSet(max_len = 50000, number_path = 500000, batch = 4, threads = 512, seed  =1999 )
# dataset = datasets.DatasetFolder(
#     root=path,
#     loader=npy_loader,
#     extensions=['.pt']
# )
# dataset = atDataset()
dataset = DataLoader(tdataset, 1000)

# dataset size is 10,000
def train_update(engine, batch):
    model.train()
    optimizer.zero_grad()
    x = batch[0]
    y = batch[1]
    y_pred = model(x)
    loss = loss_fn(y_pred[:,0], y)
    loss.backward()
    optimizer.step()
    return loss.item()

trainer = Engine(train_update)
log_interval = 100

scheduler = CosineAnnealingScheduler(optimizer, 'lr', 1e-4, 1e-6, len(dataset))
trainer.add_event_handler(Events.ITERATION_STARTED, scheduler)
timer.attach(trainer,
             start=Events.EPOCH_STARTED,
             resume=Events.ITERATION_STARTED,
             pause=Events.ITERATION_COMPLETED,
             step=Events.ITERATION_COMPLETED)    
@trainer.on(Events.ITERATION_COMPLETED)
def log_training_loss(engine):
    iter = (engine.state.iteration - 1) % len(dataset) + 1
    if iter % log_interval == 0:
        print('loss', engine.state.output, 'average time', timer.value())

# @trainer.on(Events.GET_BATCH_STARTED)
# def log_training_loss(engine):
#     print("EPOCH!!!!!!!!!!!!\n")
        
# trainer.run(dataset, max_epochs=100)
trainer.run(dataset, max_epochs=100)

Device used : cuda:0
loss 0.764177680015564 average time 0.00397009100066498
loss 1.7381147146224976 average time 0.0028936060023261236
loss 0.827140748500824 average time 0.002560490669760232
loss 1.6986442804336548 average time 0.002378168251598254
loss 0.6409973502159119 average time 0.0022879632003605364
loss 0.17690925300121307 average time 0.002230839166441001
loss 0.6499443054199219 average time 0.0021892805705179593
loss 0.3218832015991211 average time 0.0021643407492956614
loss 0.6044935584068298 average time 0.002131932221430664
loss 0.24408355355262756 average time 0.0021044413989875464
loss 0.6188947558403015 average time 0.0018604420067276806
loss 2.731745958328247 average time 0.001830730004585348
loss 0.8421614766120911 average time 0.0018336793365112194
loss 1.5340837240219116 average time 0.0018397652538260445
loss 0.5131850838661194 average time 0.0018637948015239089
loss 0.18646623194217682 average time 0.0018808750004973263
loss 0.6652289032936096 average time 0.001

State:
	iteration: 100000
	epoch: 100
	epoch_length: 1000
	max_epochs: 100
	output: 0.033259712159633636
	batch: <class 'list'>
	metrics: <class 'dict'>
	dataloader: <class 'torch.utils.data.dataloader.DataLoader'>
	seed: <class 'NoneType'>
	times: <class 'dict'>

In [36]:
import dask
dask.config.set({"dataframe.backend": "cudf"})
import dask_cudf
from dask.delayed import delayed
from dask_cuda import LocalCUDACluster
cluster = LocalCUDACluster()
from dask.distributed import Client
client = Client(cluster)
client

ModuleNotFoundError: No module named 'dask_cudf'